## Import Libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm
import timm

## Data Loading

In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ─── 1. Paths ────────────────────────────────────────────────────────────────
# DATA_DIR = "/kaggle/input/brain-tumour"
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

# ─── 2. Base Transform (no normalization) ──────────────────────────────────
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),            # scales to [0,1], shape [C,H,W]
])

# ─── 3. Create intermediate loaders to compute stats ──────────────────────
train_ds_raw = datasets.ImageFolder(root=TRAIN_DIR, transform=base_transform)
test_ds_raw  = datasets.ImageFolder(root=TEST_DIR,  transform=base_transform)

train_loader_raw = DataLoader(train_ds_raw, batch_size=32, shuffle=False, num_workers=4)
test_loader_raw  = DataLoader(test_ds_raw,  batch_size=32, shuffle=False, num_workers=4)

# ─── 4. Helper to compute mean & std ───────────────────────────────────────
def compute_mean_std(loader):
    channel_sum   = torch.zeros(3)
    channel_sqsum = torch.zeros(3)
    num_pixels    = 0

    for imgs, _ in loader:
        # imgs: [B,3,H,W]
        B, C, H, W = imgs.shape
        num_pixels += B * H * W

        channel_sum   += imgs.sum(dim=[0,2,3])
        channel_sqsum += (imgs ** 2).sum(dim=[0,2,3])

    mean = channel_sum / num_pixels
    # E[x^2] - (E[x])^2
    var  = (channel_sqsum / num_pixels) - (mean ** 2)
    std  = torch.sqrt(var)

    return mean.tolist(), std.tolist()

train_mean, train_std = compute_mean_std(train_loader_raw)
test_mean,  test_std  = compute_mean_std(test_loader_raw)

print("Train mean:", train_mean)
print("Train std: ", train_std)
print("Test mean: ", test_mean)
print("Test std:  ", test_std)

# ─── 5. Define final transforms using computed stats ───────────────────────
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=train_mean, std=train_std),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=train_mean, std=train_std),  # use train stats for test
])

# ─── 6. Reload datasets with normalization ─────────────────────────────────
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)
test_dataset  = datasets.ImageFolder(root=TEST_DIR,  transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,  num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# ─── 7. Quick Sanity Check ─────────────────────────────────────────────────
print("Classes:", train_dataset.classes)
print("Class→idx mapping:", train_dataset.class_to_idx)
print("Num train samples:", len(train_dataset))
print("Num test samples: ", len(test_dataset))

In [ ]:
# train_dataset[0]

## Data Analysis

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

# 1. Point to your Training folder
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"

# 2. Collect all image paths with labels
image_paths = []
for label in os.listdir(TRAIN_DIR):
    label_dir = os.path.join(TRAIN_DIR, label)
    if os.path.isdir(label_dir):
        for fname in os.listdir(label_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                image_paths.append((os.path.join(label_dir, fname), label))

# 3. Randomly sample 4 images
sample_paths = random.sample(image_paths, 4)

# 4. Plot them
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for ax, (img_path, label) in zip(axes, sample_paths):
    img = Image.open(img_path).convert('L')  # grayscale
    ax.imshow(img, cmap='gray')
    ax.set_title(label)
    ax.axis('off')

plt.tight_layout()
plt.show()

## Feature Extraction

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis
from PIL import Image

# ——— Manual GLCM implementation with overflow‐safe bins ———
def compute_glcm_features(arr, levels=8):
    # make sure min_val/max_val are Python scalars, not uint8
    min_val = int(arr.min())
    max_val = int(arr.max())
    # now no overflow when adding 1
    bins = np.linspace(min_val, max_val + 1, levels + 1)

    # quantize to 0…levels-1, then clip
    arr_q = np.digitize(arr, bins) - 1
    arr_q = np.clip(arr_q, 0, levels - 1)

    H, W = arr_q.shape
    glcm = np.zeros((levels, levels), dtype=np.float64)

    # horizontal co-occurrences
    for i in range(H):
        for j in range(W - 1):
            a = arr_q[i, j]
            b = arr_q[i, j + 1]
            glcm[a, b] += 1

    # symmetrize
    glcm = glcm + glcm.T

    total = glcm.sum()
    P = glcm / total if total > 0 else glcm

    i_inds, j_inds = np.indices(P.shape)
    contrast      = ((i_inds - j_inds)**2 * P).sum()
    dissimilarity = (np.abs(i_inds - j_inds) * P).sum()
    homogeneity   = (P / (1.0 + np.abs(i_inds - j_inds))).sum()
    ASM           = (P**2).sum()
    energy        = np.sqrt(ASM)

    mu_i    = (i_inds * P).sum()
    mu_j    = (j_inds * P).sum()
    sigma_i = np.sqrt(((i_inds - mu_i)**2 * P).sum())
    sigma_j = np.sqrt(((j_inds - mu_j)**2 * P).sum())

    denom = sigma_i * sigma_j
    correlation = (((i_inds - mu_i)*(j_inds - mu_j) * P).sum()) / denom \
                  if denom > 0 else 0

    return contrast, dissimilarity, homogeneity, energy, correlation, ASM

# ——— Radiomics feature extractor ———
def extract_radiomics(pil_img):
    arr = np.array(pil_img, dtype=np.float64)
    if arr.ndim == 3:
        arr = arr.mean(axis=2)          # RGB → gray
    mn, mx = arr.min(), arr.max()
    arr = (arr - mn) / (mx - mn) if mx > mn else arr

    # first‐order
    mean_intensity = arr.mean()
    std_intensity  = arr.std()
    skewness       = skew(arr.flatten())
    kurt_val       = kurtosis(arr.flatten())

    # texture via GLCM (8 levels)
    uint8_img = (arr * 255).astype(np.uint8)
    glcm_feats = compute_glcm_features(uint8_img, levels=8)

    # frequency
    fft_mag   = np.abs(np.fft.fftshift(np.fft.fft2(arr)))
    freq_mean = fft_mag.mean()
    freq_std  = fft_mag.std()

    return [
        mean_intensity, std_intensity, skewness, kurt_val,
        *glcm_feats,   # contrast, dissimilarity, homogeneity, energy, correlation, ASM
        freq_mean, freq_std
    ]

# ——— Example on your HuggingFace dataset ———
if __name__ == "__main__":
    # assume `ds` is already loaded; e.g. via `datasets.load_dataset(...)`
    # s = ds["train"][0]
    # img = s["image"]    # PIL.Image

    img = Image.open("/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing/glioma_tumor/1879.png")

    arr = np.array(img)
    print(f"Image shape: {arr.shape}, dtype: {arr.dtype}")
    print(f"Value range: {arr.min()}–{arr.max()}")

    feature_names = [
        "mean_intensity","std_intensity","skewness","kurtosis",
        "contrast","dissimilarity","homogeneity","energy",
        "correlation","ASM","freq_mean","freq_std"
    ]
    feats = extract_radiomics(img)

    print("\nRadiomic features:")
    for name, val in zip(feature_names, feats):
        print(f" • {name}: {val:.4f}")

In [ ]:
# import os
# import numpy as np
# from scipy.stats import skew, kurtosis
# from PIL import Image

# TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
# TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

# # Mapping from class name to integer label
# label_map = {
#     "glioma_tumor": 0,
#     "meningioma_tumor": 1,
#     "no_tumor": 2,
#     "pituitary_tumor": 3
# }

# # ——— Manual GLCM implementation with overflow‑safe bins ———
# def compute_glcm_features(arr, levels=8):
#     min_val = int(arr.min())
#     max_val = int(arr.max())
#     bins = np.linspace(min_val, max_val + 1, levels + 1)
#     arr_q = np.clip(np.digitize(arr, bins) - 1, 0, levels - 1)
#     H, W = arr_q.shape
#     glcm = np.zeros((levels, levels), dtype=np.float64)
#     for i in range(H):
#         for j in range(W - 1):
#             glcm[arr_q[i, j], arr_q[i, j + 1]] += 1
#     glcm = glcm + glcm.T
#     total = glcm.sum()
#     P = glcm / total if total > 0 else glcm
#     i_inds, j_inds = np.indices(P.shape)
#     contrast      = ((i_inds - j_inds)**2 * P).sum()
#     dissimilarity = (np.abs(i_inds - j_inds) * P).sum()
#     homogeneity   = (P / (1.0 + np.abs(i_inds - j_inds))).sum()
#     ASM           = (P**2).sum()
#     energy        = np.sqrt(ASM)
#     mu_i    = (i_inds * P).sum()
#     mu_j    = (j_inds * P).sum()
#     sigma_i = np.sqrt(((i_inds - mu_i)**2 * P).sum())
#     sigma_j = np.sqrt(((j_inds - mu_j)**2 * P).sum())
#     denom = sigma_i * sigma_j
#     correlation = (((i_inds - mu_i)*(j_inds - mu_j) * P).sum()) / denom if denom > 0 else 0
#     return contrast, dissimilarity, homogeneity, energy, correlation, ASM

# # ——— Radiomics feature extractor ———
# def extract_radiomics(pil_img):
#     arr = np.array(pil_img, dtype=np.float64)
#     if arr.ndim == 3:
#         arr = arr.mean(axis=2)
#     mn, mx = arr.min(), arr.max()
#     arr = (arr - mn) / (mx - mn) if mx > mn else arr
#     mean_intensity = arr.mean()
#     std_intensity  = arr.std()
#     skewness       = skew(arr.flatten())
#     kurt_val       = kurtosis(arr.flatten())
#     uint8_img = (arr * 255).astype(np.uint8)
#     glcm_feats = compute_glcm_features(uint8_img, levels=8)
#     fft_mag   = np.abs(np.fft.fftshift(np.fft.fft2(arr)))
#     freq_mean = fft_mag.mean()
#     freq_std  = fft_mag.std()
#     return [
#         mean_intensity, std_intensity, skewness, kurt_val,
#         *glcm_feats,
#         freq_mean, freq_std
#     ]

# def load_radiomics_from_dir(base_dir):
#     X, y = [], []
#     classes = sorted(os.listdir(base_dir))
#     for label in classes:
#         class_dir = os.path.join(base_dir, label)
#         if not os.path.isdir(class_dir):
#             continue
#         for fname in os.listdir(class_dir):
#             if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".bmp")):
#                 continue
#             path = os.path.join(class_dir, fname)
#             try:
#                 img = Image.open(path).convert("RGB")
#                 feats = extract_radiomics(img)
#                 X.append(feats)
#                 y.append(label_map[label])  # map string to integer
#             except Exception as e:
#                 print(f"Skipped {path}: {e}")
#     return np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)

# if __name__ == "__main__":
#     # Load train
#     X_train, y_train = load_radiomics_from_dir(TRAIN_DIR)
#     print(f"Loaded {len(y_train)} training samples, feature vector length = {X_train.shape[1]}")
#     # Load test
#     X_test, y_test = load_radiomics_from_dir(TEST_DIR)
#     print(f"Loaded {len(y_test)} testing samples, feature vector length = {X_test.shape[1]}")
#     # Save to .npz
#     np.savez(
#         "radiomics_dataset.npz",
#         X_train=X_train,
#         y_train=y_train,
#         X_test=X_test,
#         y_test=y_test
#     )
#     print("Saved radiomics_dataset.npz with integer labels")

In [ ]:
data = np.load("radiomics_dataset.npz", allow_pickle=True)
X_train, y_train = data["X_train"], data["y_train"]
X_test,  y_test  = data["X_test"],  data["y_test"]
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape:  {X_test.shape}, y_test shape:  {y_test.shape}")

In [ ]:
y_test

In [ ]:
import numpy as np

# 1. Load the .npz file
rad = np.load('/home/mhs/thesis/radiomics_dataset.npz')

# 2. Extract train/test splits
X_train_rad, y_train = rad['X_train'], rad['y_train']
X_test_rad,  y_test  = rad['X_test'],  rad['y_test']

# 3. (Optional) Inspect shapes
print("X_train_rad shape:", X_train_rad.shape)
print("y_train shape:   ", y_train.shape)
print("X_test_rad shape: ", X_test_rad.shape)
print("y_test shape:    ", y_test.shape)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from xgboost import XGBClassifier

import numpy as np

# 1. Load the .npz file
rad = np.load('/home/mhs/thesis/radiomics_dataset.npz')

# 2. Extract train/test splits
X_train_rad, y_train = rad['X_train'], rad['y_train']
X_test_rad,  y_test  = rad['X_test'],  rad['y_test']

# 3. (Optional) Inspect shapes
print("X_train_rad shape:", X_train_rad.shape)
print("y_train shape:   ", y_train.shape)
print("X_test_rad shape: ", X_test_rad.shape)
print("y_test shape:    ", y_test.shape)


# --- 2) Radiomics-only classifiers ---
# a) Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_rad, y_train)
y_pred_rf = rf_clf.predict(X_test_rad)
print("Random Forest on radiomics only:")
print("  Test Acc:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

xg = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xg.fit(X_train_rad, y_train)
y_pred_rf = xg.predict(X_test_rad)
print("XGBoost on radiomics only:")
print("  Test Acc:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

In [ ]:
# !pip install pykan pytorch-tabnet torch scikit-learn numpy

In [ ]:
# pip install pykan

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from pytorch_tabnet.tab_model import TabNetClassifier
from kan import KAN
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# 1. Load the .npz file
rad = np.load('/home/mhs/thesis/radiomics_dataset.npz')

# 2. Extract train/test splits
X_train_rad, y_train = rad['X_train'], rad['y_train']
X_test_rad, y_test = rad['X_test'], rad['y_test']

# 3. Inspect shapes
print("X_train_rad shape:", X_train_rad.shape)
print("y_train shape:   ", y_train.shape)
print("X_test_rad shape: ", X_test_rad.shape)
print("y_test shape:    ", y_test.shape)

# --- 2) Radiomics-only classifiers ---

# a) KAN (Kolmogorov-Arnold Network)
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_rad, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_rad, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Initialize KAN model
# Define the architecture: [input_dim, hidden_dim, output_dim]
input_dim = X_train_rad.shape[1]
num_classes = len(np.unique(y_train))
kan_model = KAN(width=[input_dim, 64, num_classes], grid=5, k=3)

# Training function for KAN
def train_kan(model, X, y, epochs=100, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in tqdm(range(epochs), desc="KAN Training"):
        model.train()
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

# Train KAN model
train_kan(kan_model, X_train_tensor, y_train_tensor, epochs=100, lr=0.001)

# Evaluate KAN model
kan_model.eval()
with torch.no_grad():
    y_pred_kan = kan_model(X_test_tensor).argmax(dim=1).numpy()

print("KAN on radiomics only:")
print("  Test Acc:", accuracy_score(y_test, y_pred_kan))
print(classification_report(y_test, y_pred_kan))

# b) TabNet
# Initialize TabNet model
tabnet_clf = TabNetClassifier(
    n_d=8,  # Dimension of the prediction layer
    n_a=8,  # Dimension of the attention layer
    n_steps=3,  # Number of steps in the architecture
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.02),
    verbose=1
)

# Fit TabNet model
tabnet_clf.fit(
    X_train=X_train_rad,
    y_train=y_train,
    eval_set=[(X_test_rad, y_test)],
    eval_name=['test'],
    eval_metric=['accuracy'],
    max_epochs=100,
    patience=20,  # Early stopping after 20 epochs without improvement
    batch_size=1024,
    virtual_batch_size=128
)

# Predict with TabNet
y_pred_tabnet = tabnet_clf.predict(X_test_rad)

print("TabNet on radiomics only:")
print("  Test Acc:", accuracy_score(y_test, y_pred_tabnet))
print(classification_report(y_test, y_pred_tabnet))

X_train_rad shape: (13927, 12)
y_train shape:    (13927,)
X_test_rad shape:  (3961, 12)
y_test shape:     (3961,)
checkpoint directory created: ./model
saving model version 0.0


KAN Training: 100%|██████████| 100/100 [01:19<00:00,  1.26it/s]
/home/mhs/miniforge3/envs/thesis/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


KAN on radiomics only:
  Test Acc: 0.4165614743751578
              precision    recall  f1-score   support

           0       0.45      0.82      0.58      1208
           1       0.36      0.31      0.33       930
           2       0.83      0.17      0.28       831
           3       0.29      0.23      0.26       992

    accuracy                           0.42      3961
   macro avg       0.48      0.38      0.36      3961
weighted avg       0.47      0.42      0.38      3961

epoch 0  | loss: 1.45014 | test_accuracy: 0.29841 |  0:00:00s
epoch 1  | loss: 1.0384  | test_accuracy: 0.33754 |  0:00:01s
epoch 2  | loss: 0.94764 | test_accuracy: 0.20298 |  0:00:01s
epoch 3  | loss: 0.90773 | test_accuracy: 0.18581 |  0:00:01s
epoch 4  | loss: 0.87215 | test_accuracy: 0.28427 |  0:00:49s
epoch 5  | loss: 0.83824 | test_accuracy: 0.29891 |  0:00:02s
epoch 6  | loss: 0.8281  | test_accuracy: 0.3176  |  0:00:02s
epoch 7  | loss: 0.81142 | test_accuracy: 0.36354 |  0:00:03s
epoch 8  | loss

/home/mhs/miniforge3/envs/thesis/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet on radiomics only:
  Test Acc: 0.7331481949002777
              precision    recall  f1-score   support

           0       0.77      0.66      0.71      1208
           1       0.60      0.58      0.59       930
           2       0.77      0.87      0.82       831
           3       0.77      0.85      0.81       992

    accuracy                           0.73      3961
   macro avg       0.73      0.74      0.73      3961
weighted avg       0.73      0.73      0.73      3961



## CNN Model

In [ ]:
# 0) Common imports
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report

# ─── 1) Paths & Transforms ────────────────────────────────────────────
# DATA_DIR  = "/kaggle/input/brain-tumour"
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

# # (Use your previously computed means & stds here)
# train_mean = [0.32, 0.28, 0.30]
# train_std  = [0.12, 0.11, 0.13]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

# ─── 2) Datasets & Loaders ───────────────────────────────────────────
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)
test_dataset  = datasets.ImageFolder(root=TEST_DIR,  transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, 
                          num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, 
                          num_workers=4, pin_memory=True)

# Number of classes
num_classes = len(train_dataset.classes)

# ─── 3) SimpleCNN Definition ─────────────────────────────────────────
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        # 224→112→56→28 after three pools
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*28*28, 256), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(self.features(x))

# ─── 4) Setup ─────────────────────────────────────────────────────────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = SimpleCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# ─── 5) Training & Evaluation ────────────────────────────────────────
for epoch in range(1, 11):
    # -- train
    model.train()
    for imgs, lbls in train_loader:
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad()
        loss = criterion(model(imgs), lbls)
        loss.backward()
        optimizer.step()

    # -- eval
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, lbls in test_loader:
            imgs = imgs.to(device)
            preds = model(imgs).argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(lbls.numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch:2d}/10 — Test Acc: {acc:.4f}")

# ─── 6) Final Report ───────────────────────────────────────────────────
print("\nFinal CNN classification report:")
print(classification_report(all_labels, all_preds, 
                            target_names=train_dataset.classes, digits=4))

## Hyper Tunning

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

# ─── 1) Paths & MRI-specific normalization stats ───────────────────────────
# DATA_DIR  = "/kaggle/input/brain-tumour"
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

# # (Use your previously computed means & stds here)
# train_mean = [0.32, 0.28, 0.30]
# train_std  = [0.12, 0.11, 0.13]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

# ─── 3) Globals ────────────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# determine number of classes from your folder structure
num_classes = len(next(os.walk(TRAIN_DIR))[1])

# ─── 4) SimpleCNN with tunable dropout ────────────────────────────────────
class SimpleCNN(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,  32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        # 224→112→56→28 after 3 pools
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*28*28, 256),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# ─── 5) Optuna objective ──────────────────────────────────────────────────
def objective(trial):
    # 5.1) Hyperparameters to tune
    lr           = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.7)
    batch_size   = trial.suggest_categorical("batch_size", [16, 32, 64])
    momentum     = trial.suggest_float("momentum", 0.5, 0.99)

    # 5.2) DataLoaders with those batch sizes
    train_loader = DataLoader(
        datasets.ImageFolder(TRAIN_DIR, transform=train_transform),
        batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True
    )
    test_loader = DataLoader(
        datasets.ImageFolder(TEST_DIR, transform=test_transform),
        batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True
    )

    # 5.3) Model, loss & optimizer
    model     = SimpleCNN(num_classes, dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    # 5.4) Training loop (10 epochs)
    for _ in range(10):
        model.train()
        for imgs, lbls in train_loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            optimizer.zero_grad()
            loss = criterion(model(imgs), lbls)
            loss.backward()
            optimizer.step()

    # 5.5) Evaluation on test set
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, lbls in test_loader:
            imgs = imgs.to(device)
            preds = model(imgs).argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(lbls.numpy())

    return accuracy_score(all_labels, all_preds)

# ─── 6) Run the Optuna study ───────────────────────────────────────────────
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    print("Best accuracy:", study.best_value)
    print("Best hyperparameters:", study.best_trial.params)

## Further Hyper-tunning

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets

import optuna
from optuna.pruners import MedianPruner
from tqdm import tqdm

# -----------------------------------------------------------------------------
# 0) Paths and number of classes
# -----------------------------------------------------------------------------
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

num_classes = len(next(os.walk(TRAIN_DIR))[1])
print(f"Detected {num_classes} classes.")

# -----------------------------------------------------------------------------
# 1) Data transforms
# -----------------------------------------------------------------------------
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

# -----------------------------------------------------------------------------
# 2) Datasets and train/validation split
# -----------------------------------------------------------------------------
full_train = datasets.ImageFolder(root=TRAIN_DIR, transform=transform)
test_ds    = datasets.ImageFolder(root=TEST_DIR,  transform=transform)

train_len = int(0.8 * len(full_train))
val_len   = len(full_train) - train_len
train_ds, val_ds = random_split(full_train, [train_len, val_len])

def make_loader(dataset, batch_size, shuffle):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                      num_workers=4, pin_memory=True)

# -----------------------------------------------------------------------------
# 3) Define the Optuna-powered CNN
# -----------------------------------------------------------------------------
class OptunaCNN(nn.Module):
    def __init__(self, trial, num_classes, input_shape):
        super().__init__()
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()

        in_ch = input_shape[0]
        n_conv = trial.suggest_int('n_conv_layers', 1, 4)
        for i in range(n_conv):
            out_ch = trial.suggest_categorical(f"conv{i}_out", [16, 32, 64, 128])
            k      = trial.suggest_int(f"conv{i}_k", 3, 7, step=2)
            p      = trial.suggest_int(f"conv{i}_pad", 0, k // 2)
            self.convs.append(nn.Conv2d(in_ch, out_ch, kernel_size=k, padding=p))

            norm_t = trial.suggest_categorical(f"conv{i}_norm", ['none', 'batch', 'layer'])
            if norm_t == 'batch':
                self.norms.append(nn.BatchNorm2d(out_ch))
            elif norm_t == 'layer':
                self.norms.append(nn.GroupNorm(1, out_ch))
            else:
                self.norms.append(None)

            in_ch = out_ch

        self.pool = nn.MaxPool2d(2)

        with torch.no_grad():
            x = torch.zeros(1, *input_shape)
            for conv, norm in zip(self.convs, self.norms):
                x = conv(x)
                if norm: x = norm(x)
                x = F.relu(x)
                x = self.pool(x)
            n_flat = x.numel()

        self.fcs = nn.ModuleList()
        n_fc = trial.suggest_int('n_fc_layers', 1, 3)
        in_feat = n_flat
        for j in range(n_fc):
            out_f = trial.suggest_int(f"fc{j}_units", 32, 512, step=32)
            self.fcs.append(nn.Linear(in_feat, out_f))
            in_feat = out_f

        self.output    = nn.Linear(in_feat, num_classes)
        self.dropout_p = trial.suggest_uniform('dropout', 0.0, 0.5)
        self.act_name  = trial.suggest_categorical('activation', ['relu', 'leaky_relu', 'elu'])

    def forward(self, x):
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x)
            if norm: x = norm(x)
            x = getattr(F, self.act_name)(x)  
            x = self.pool(x)

        x = x.view(x.size(0), -1)
        for fc in self.fcs:
            x = fc(x)
            x = getattr(F, self.act_name)(x)
            x = F.dropout(x, p=self.dropout_p, training=self.training)

        return self.output(x)

# -----------------------------------------------------------------------------
# 4) Optuna objective with weight_decay
# -----------------------------------------------------------------------------
def objective(trial):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # hyperparameters
    batch_size   = trial.suggest_categorical('batch_size', [16, 32, 64])
    lr           = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    opt_name     = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    epochs       = trial.suggest_int('epochs', 10, 50)
    patience     = trial.suggest_int('patience', 3, 7)

    # DataLoaders
    train_loader = make_loader(train_ds, batch_size, shuffle=True)
    val_loader   = make_loader(val_ds,   batch_size, shuffle=False)

    # build model
    sample_img, _ = next(iter(train_loader))
    input_shape   = sample_img.shape[1:]
    model = OptunaCNN(trial, num_classes=num_classes, input_shape=input_shape).to(device)

    # optimizer with weight_decay
    if opt_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif opt_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        momentum  = trial.suggest_uniform('momentum', 0.5, 0.9)
        optimizer = optim.SGD(model.parameters(), lr=lr,
                              momentum=momentum,
                              weight_decay=weight_decay)

    criterion = nn.CrossEntropyLoss()
    best_loss = float('inf')
    no_improve = 0

    for epoch in range(epochs):
        model.train()
        for imgs, lbls in train_loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            optimizer.zero_grad()
            loss = criterion(model(imgs), lbls)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for imgs, lbls in val_loader:
                imgs, lbls = imgs.to(device), lbls.to(device)
                val_loss += criterion(model(imgs), lbls).item()
        avg_loss = val_loss / len(val_loader)

        trial.report(avg_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                break

    return best_loss

# -----------------------------------------------------------------------------
# 5) Run the Optuna study
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize", pruner=MedianPruner())
    study.optimize(objective, n_trials=50)

    print("Best trial:")
    print(f"  Loss:   {study.best_value:.4f}")
    print("  Params:")
    for key, value in study.best_params.items():
        print(f"    {key}: {value}")

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

# -------------------------------------------------------------------
# 0) Paths and num_classes
# -------------------------------------------------------------------
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

num_classes = len(next(os.walk(TRAIN_DIR))[1])
print(f"Detected {num_classes} classes.")

# -------------------------------------------------------------------
# 1) Transforms & DataLoaders
# -------------------------------------------------------------------
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

train_ds = datasets.ImageFolder(TRAIN_DIR, transform=transform)
test_ds  = datasets.ImageFolder(TEST_DIR,  transform=transform)

batch_size   = 16
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                          num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False,
                          num_workers=4, pin_memory=True)

# -------------------------------------------------------------------
# 2) FixedCNN with dynamic flat_dim
# -------------------------------------------------------------------
conv_params = [
    (128, 3, 0, 'none'),
    ( 16, 7, 3, 'layer'),
    (128, 7, 3, 'batch'),
    ( 16, 7, 3, 'batch'),
]
fc_units  = [512]
dropout_p = 0.05406253916379472

class FixedCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # build conv feature extractor
        layers = []
        in_ch = 3
        for out_ch, k, p, norm in conv_params:
            layers.append(nn.Conv2d(in_ch, out_ch, kernel_size=k, padding=p))
            if norm == 'batch':
                layers.append(nn.BatchNorm2d(out_ch))
            elif norm == 'layer':
                layers.append(nn.GroupNorm(1, out_ch))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.MaxPool2d(2))
            in_ch = out_ch

        self.feature_extractor = nn.Sequential(*layers)

        # determine flattened dimension automatically
        with torch.no_grad():
            dummy = torch.zeros(1, 3, 224, 224)
            feat  = self.feature_extractor(dummy)
            flat_dim = feat.view(1, -1).size(1)

        # build classifier head
        fc_layers = []
        in_feat = flat_dim
        for out_feat in fc_units:
            fc_layers.append(nn.Linear(in_feat, out_feat))
            fc_layers.append(nn.ReLU(inplace=True))
            fc_layers.append(nn.Dropout(dropout_p))
            in_feat = out_feat

        # final output layer
        fc_layers.append(nn.Linear(in_feat, num_classes))
        self.classifier = nn.Sequential(*fc_layers)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# -------------------------------------------------------------------
# 3) Instantiate, optimizer, criterion
# -------------------------------------------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = FixedCNN(num_classes).to(device)
optimizer = optim.RMSprop(
    model.parameters(),
    lr=8.742429747329988e-05,
    weight_decay=0.0008288643868674648
)
criterion = nn.CrossEntropyLoss()

# -------------------------------------------------------------------
# 4) Training loop
# -------------------------------------------------------------------
epochs = 19
for epoch in range(1, epochs + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc  = 100. * correct / total
    print(f"Epoch {epoch}: Loss = {epoch_loss:.4f}, Acc = {epoch_acc:.2f}%")

# -------------------------------------------------------------------
# 5) Final evaluation on test set
# -------------------------------------------------------------------
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(dim=1).cpu().tolist()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().tolist())

test_acc = accuracy_score(all_labels, all_preds) * 100
print(f"\nTest Accuracy: {test_acc:.2f}%")
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=train_ds.classes))


## KNN Model

In [ ]:
# !pip install pykan

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# ─── 1) Paths & Transforms ────────────────────────────────────────────
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])

# ─── 2) Datasets & Loaders ───────────────────────────────────────────
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)
test_dataset  = datasets.ImageFolder(root=TEST_DIR,  transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,
                          num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False,
                          num_workers=4, pin_memory=True)

num_classes = len(train_dataset.classes)


# ─── 3) Your KAN Backbone Stub ────────────────────────────────────────
# Replace this with your actual KAN implementation.
class KANBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        # Example: you might have conv layers + attention blocks here
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # ... more layers / attention ...
            nn.Conv2d(64, 512, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)   # [B, 512, H, W]


# ─── 4) Full KANModel with Pooling + Classifier ───────────────────────
class KANModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone    = KANBackbone()
        self.global_pool = nn.AdaptiveAvgPool2d((1,1))  # → [B, 512, 1, 1]
        self.classifier  = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.backbone(x)        # [B, 512, H, W]
        x = self.global_pool(x)     # [B, 512, 1, 1]
        x = torch.flatten(x, 1)     # [B, 512]
        logits = self.classifier(x) # [B, num_classes]
        return logits


# ─── 5) Setup device, model, loss, optimizer ─────────────────────────
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = KANModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)


# ─── 6) Training & Eval Routines ─────────────────────────────────────
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)

    return running_loss/total, correct/total

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Eval", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss    = criterion(outputs, labels)

            running_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

    return running_loss/total, correct/total


# ─── 7) Main Loop with Checkpointing ──────────────────────────────────
num_epochs = 25
best_acc   = 0.0

for epoch in range(1, num_epochs+1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss,   val_acc   = evaluate(model, test_loader,  criterion, device)

    print(f"Epoch {epoch:02d} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f" Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    # save best
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_kan_model.pth")

print(f"Best validation accuracy: {best_acc:.4f}")

In [ ]:
import os
import torch
import torch.nn as nn
import torch.backends.cudnn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

# ─── 1) Enable cuDNN autotuner for faster convolutions ────────────────
torch.backends.cudnn.benchmark = True

# ─── 2) Paths & Transforms ────────────────────────────────────────────
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

train_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])

# ─── 3) Datasets & Loaders ───────────────────────────────────────────
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)
test_dataset  = datasets.ImageFolder(root=TEST_DIR,  transform=test_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

num_classes = len(train_dataset.classes)

# ─── 4) KAN Backbone Stub (replace with your real implementation) ─────
class KANBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        # Example conv + attention stub; insert your KAN layers here
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # ... more conv/attention blocks ...
            nn.Conv2d(64, 512, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)  # → [B, 512, H, W]

# ─── 5) Full Model with Global Pooling + Classifier ───────────────────
class KANModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone    = KANBackbone()
        self.global_pool = nn.AdaptiveAvgPool2d((1,1))  # → [B, 512, 1, 1]
        self.classifier  = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.backbone(x)        # [B, 512, H, W]
        x = self.global_pool(x)     # [B, 512, 1, 1]
        x = torch.flatten(x, 1)     # [B, 512]
        return self.classifier(x)   # [B, num_classes]

# ─── 6) Setup Device, Model, Loss, Optimizer, AMP ────────────────────
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = KANModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scaler    = GradScaler()

# ─── 7) Training & Evaluation Routines ───────────────────────────────
def train_one_epoch(model, loader, criterion, optimizer, device, scaler):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast():  # mixed-precision forward
            outputs = model(imgs)
            loss    = criterion(outputs, labels)

        scaler.scale(loss).backward()  # scaled backward
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)

    return running_loss / total, correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad(), autocast():  # mixed-precision eval
        for imgs, labels in tqdm(loader, desc="Eval", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss    = criterion(outputs, labels)

            running_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

    return running_loss / total, correct / total

# ─── 8) Main Loop with Checkpointing & Interrupt Handling ────────────
num_epochs = 10
best_acc   = 0.0

try:
    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device, scaler
        )
        val_loss, val_acc = evaluate(model, test_loader, criterion, device)

        print(f"Epoch {epoch:02d} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_kan_model.pth")

except KeyboardInterrupt:
    print("\n🛑 Training interrupted. Saving checkpoint to 'interrupted_kan_model.pth'")
    torch.save(model.state_dict(), "interrupted_kan_model.pth")
    raise

print(f"✅ Best validation accuracy: {best_acc:.4f}")


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report
from kan import KAN
import time

class KANClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        print("Initializing feature extractor...")
        
        # 1. Feature Extractor
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(mobilenet.children())[:-1])
        
        # 2. Feature Processing
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.2)
        self.fc_reduce = nn.Linear(1280, 256)
        
        # 3. KAN Layer
        self.kan = KAN(
            width=[256, 64, num_classes],  # [input_dim, hidden_dim, output_dim]
            grid=3,
            k=2,
            noise_scale=0.1,
            base_fun=torch.nn.ReLU(),
            grid_eps=1.0,
            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        )
        
    def forward(self, x):
        # 1. Extract features with shape checking
        if x.dim() != 4:
            raise ValueError(f"Expected 4D input (BxCxHxW), got {x.dim()}D")
        
        # 2. Pass through feature extractor
        features = self.feature_extractor(x)  # [B, 1280, H', W']
        
        # 3. Global pooling and reshape
        features = self.pool(features)        # [B, 1280, 1, 1]
        features = features.view(x.size(0), -1)  # [B, 1280]
        
        # 4. Dimension reduction and preprocessing
        features = self.dropout(features)
        features = self.fc_reduce(features)   # [B, 256]
        
        # 5. KAN expects [batch_size, feature_dim]
        # No need for additional unsqueeze since KAN handles the reshaping internally
        output = self.kan(features)           # [B, num_classes]
        
        return output

# Training settings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize model and move to device
model = KANClassifier(num_classes).to(device)

# Optimizer with lower learning rate for stability
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch {epoch}: Loss = {avg_loss:.4f}, Acc = {accuracy:.2f}%')

    # Evaluation phase
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, lbls in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(lbls.numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch:2d}/10 — Test Acc: {acc:.4f}")

# Final Report
print("\nFinal KAN classification report:")
print(classification_report(all_labels, all_preds, 
                          target_names=train_dataset.classes, 
                          digits=4))

In [ ]:
import os
import torch
import torch.nn as nn
import torch.backends.cudnn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

# ─── 1) Enable cuDNN autotuner for faster convolutions ────────────────
torch.backends.cudnn.benchmark = True

# ─── 2) Paths & Transforms ────────────────────────────────────────────
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

train_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])

# ─── 3) Datasets & Loaders ───────────────────────────────────────────
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)
test_dataset  = datasets.ImageFolder(root=TEST_DIR,  transform=test_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

num_classes = len(train_dataset.classes)


# ─── 4) Pretrained ResNet-50 Backbone ─────────────────────────────────
class PretrainedBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        # load a ResNet50 pretrained on ImageNet
        backbone = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        # optionally freeze all layers:
        # for param in backbone.parameters():
        #     param.requires_grad = False

        # cut off the final fc & avgpool:
        self.features = nn.Sequential(*list(backbone.children())[:-2])
        self.out_channels = 2048

    def forward(self, x):
        return self.features(x)  # [B, 2048, H', W']


# ─── 5) Full Model with Pretrained Backbone ───────────────────────────
class KANModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone    = PretrainedBackbone()
        self.global_pool = nn.AdaptiveAvgPool2d((1,1))  # → [B, 2048, 1, 1]
        self.classifier  = nn.Linear(self.backbone.out_channels, num_classes)

    def forward(self, x):
        x = self.backbone(x)        # [B, 2048, H, W]
        x = self.global_pool(x)     # [B, 2048, 1, 1]
        x = torch.flatten(x, 1)     # [B, 2048]
        return self.classifier(x)   # [B, num_classes]


# ─── 6) Setup Device, Model, Loss, Optimizer, AMP ────────────────────
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = KANModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scaler    = GradScaler()

# ─── 7) Training & Evaluation Routines ───────────────────────────────
def train_one_epoch(model, loader, criterion, optimizer, device, scaler):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast():
            outputs = model(imgs)
            loss    = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)

    return running_loss / total, correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad(), autocast():
        for imgs, labels in tqdm(loader, desc="Eval", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss    = criterion(outputs, labels)

            running_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

    return running_loss / total, correct / total


# ─── 8) Main Loop with Checkpointing & Interrupt Handling ────────────
num_epochs = 25
best_acc   = 0.0

try:
    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device, scaler
        )
        val_loss, val_acc = evaluate(model, test_loader, criterion, device)

        print(f"Epoch {epoch:02d} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_kan_model.pth")

except KeyboardInterrupt:
    print("\n🛑 Training interrupted. Saving checkpoint to 'interrupted_kan_model.pth'")
    torch.save(model.state_dict(), "interrupted_kan_model.pth")
    raise

print(f"✅ Best validation accuracy: {best_acc:.4f}")


## Pre-trained Model

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from tqdm import tqdm
from sklearn.metrics import classification_report

# 1) Reproducibility + device
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2) Paths
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

# 3) Custom label mapping
label_map = {
    "glioma_tumor":     0,
    "meningioma_tumor": 1,
    "no_tumor":         2,
    "pituitary_tumor":  3
}

# 4) Transforms: PIL grayscale → Resize → Tensor → repeat → Normalize
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),            # single-channel PIL
    transforms.Resize((224, 224)),                         # resize while PIL
    transforms.ToTensor(),                                 # now a tensor [1×224×224]
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),        # expand → [3×224×224]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],       # ImageNet stats
                         std=[0.229, 0.224, 0.225])
])

# 5) Subclass ImageFolder to apply your label_map
class MappedImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        img, orig_idx = super().__getitem__(index)
        classname = self.classes[orig_idx]
        mapped_label = label_map[classname]
        return img, mapped_label

# 6) Datasets & train/val split
full_train = MappedImageFolder(root=TRAIN_DIR, transform=transform)
test_ds    = MappedImageFolder(root=TEST_DIR,  transform=transform)

train_len = int(0.8 * len(full_train))
val_len   = len(full_train) - train_len
train_ds, val_ds = random_split(full_train, [train_len, val_len])

# 7) Dataloaders
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                          num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False,
                          num_workers=4, pin_memory=True)

# 8) Number of classes
num_classes = len(label_map)
print(f"Using {num_classes} classes with mapping: {label_map}")

# 9) Model: fine-tuned ResNet50
class ResNet50Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = models.resnet50(pretrained=True)
        in_feats = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_feats, num_classes)
    
    def forward(self, x):
        return self.backbone(x)

model = ResNet50Model(num_classes).to(device)

# 10) Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 11) Training and evaluation functions
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, labels in tqdm(loader, desc="Train"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / len(loader), 100. * correct / total

def eval_epoch(model, loader):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Eval"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
    return 100. * correct / total, all_preds, all_labels

# 12) Training loop
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_acc, _, _ = eval_epoch(model, val_loader)
    print(f"Epoch {epoch}/{num_epochs} — "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Acc: {val_acc:.2f}%")
    print("-" * 60)

# 13) Final test evaluation
test_acc, preds, labels = eval_epoch(model, test_loader)
print(f"\nTest Accuracy: {test_acc:.2f}%")
print("\nClassification Report:")
print(classification_report(labels, preds, target_names=list(label_map.keys())))

# 14) Save the fine-tuned model
torch.save(model.state_dict(), "tumor_resnet50_mapped.pth")

## Fusion Models

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# 1) Reproducibility & device
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2) Paths to your tumor MRI folders
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR  = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"

# 3) Custom label mapping
label_map = {
    "glioma_tumor":     0,
    "meningioma_tumor": 1,
    "no_tumor":         2,
    "pituitary_tumor":  3
}

# 4) Transforms: PIL grayscale → Resize → Tensor → repeat → Normalize
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),    # load as single‐channel PIL image
    transforms.Resize((224, 224)),                  # resize while still PIL
    transforms.ToTensor(),                          # to tensor [1×224×224]
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)), # expand → [3×224×224]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # ImageNet statistics
])

# 5) Subclass ImageFolder to apply your custom label_map
class MappedImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        img, orig_idx = super().__getitem__(index)
        classname     = self.classes[orig_idx]
        mapped_label  = label_map[classname]
        return img, mapped_label

# 6) Build datasets & split train→val
full_train = MappedImageFolder(root=TRAIN_DIR, transform=transform)
test_ds    = MappedImageFolder(root=TEST_DIR,  transform=transform)

train_len = int(0.8 * len(full_train))
val_len   = len(full_train) - train_len
train_ds, val_ds = random_split(full_train, [train_len, val_len])

# 7) DataLoaders
batch_size   = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                          num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False,
                          num_workers=4, pin_memory=True)

# 8) Number of classes
num_classes = len(label_map)
print(f"Detected {num_classes} classes: {full_train.classes}")

# 9) Model Definitions
class ResNet50Classifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = models.resnet50(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)
    def forward(self, x):
        return self.resnet(x)

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*28*28,256), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

class FusionModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        base = models.resnet50(pretrained=True)
        # ResNet50 features (drop the final fc)
        self.resnet_feat = nn.Sequential(*list(base.children())[:-1])
        # Simple CNN features
        self.cnn_feat = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten()
        )
        # determine cnn feature dim
        with torch.no_grad():
            dummy = torch.zeros(1,3,224,224)
            cnn_dim = self.cnn_feat(dummy).shape[1]
        # fusion classifier
        self.classifier = nn.Sequential(
            nn.Linear(2048 + cnn_dim, 512),
            nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        f1 = self.resnet_feat(x).view(x.size(0), -1)
        f2 = self.cnn_feat(x)
        return self.classifier(torch.cat([f1, f2], dim=1))

# 10) Training & evaluation utilities
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        total_correct += preds.eq(labels).sum().item()
        total_samples += imgs.size(0)
    return total_loss/total_samples, total_correct/total_samples

def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1).cpu().tolist()
            all_preds.extend(preds)
            all_labels.extend(labels.tolist())
    acc    = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=list(label_map.keys()))
    return acc, report

# 11) Train & compare models
models_to_train = {
    'ResNet50':  ResNet50Classifier(num_classes),
    'SimpleCNN': SimpleCNN(num_classes),
    'Fusion':    FusionModel(num_classes)
}

criterion  = nn.CrossEntropyLoss()
num_epochs = 10
results    = {}

for name, model in models_to_train.items():
    print(f"\n=== Training {name} for {num_epochs} epochs ===")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(1, num_epochs+1):
        loss, acc = train_epoch(model, train_loader, criterion, optimizer)
        print(f"{name} Epoch {epoch}/{num_epochs} — Loss: {loss:.4f}, Acc: {acc:.4f}")

    test_acc, test_report = evaluate(model, test_loader)
    results[name] = (test_acc, test_report)
    print(f"\n{name} Final Test Acc: {test_acc:.4f}")
    print(test_report)

# 12) Summary of results
print("\n=== Summary of Test Accuracies ===")
for name, (acc, _) in results.items():
    print(f"{name}: {acc:.4f}")

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import uuid

# Paths and device
TRAIN_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Training"
TEST_DIR = "/home/mhs/thesis/Brain MRI ND-5 Dataset/tumordata/Testing"
RAD_NPZ = "/home/mhs/thesis/radiomics_dataset.npz"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load radiomics data
rad = np.load(RAD_NPZ)
X_train_rad, y_train = rad['X_train'], rad['y_train']
X_test_rad, y_test = rad['X_test'], rad['y_test']
num_classes = len(np.unique(y_train))
print(f"Detected {num_classes} classes in radiomics data.")
print("X_train_rad shape:", X_train_rad.shape)
print("X_test_rad shape:", X_test_rad.shape)

# Transforms for images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset to pair images with radiomics features
class PairedDataset(Dataset):
    def __init__(self, image_dataset, radiomics_features, labels):
        self.image_dataset = image_dataset
        self.radiomics_features = radiomics_features
        self.labels = labels
        assert len(image_dataset) == len(radiomics_features) == len(labels), "Mismatched dataset lengths"

    def __len__(self):
        return len(self.image_dataset)

    def __getitem__(self, idx):
        img, img_label = self.image_dataset[idx]
        rad_features = torch.tensor(self.radiomics_features[idx], dtype=torch.float32)
        label = self.labels[idx]
        return img, rad_features, label

# Load image datasets
train_image_ds = datasets.ImageFolder(TRAIN_DIR, transform=transform)
test_image_ds = datasets.ImageFolder(TEST_DIR, transform=transform)

# Create paired datasets
train_dataset = PairedDataset(train_image_ds, X_train_rad, y_train)
test_dataset = PairedDataset(test_image_ds, X_test_rad, y_test)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# CNN Model (same as provided)
conv_params = [
    (128, 3, 0, 'none'),
    (16, 7, 3, 'layer'),
    (128, 7, 3, 'batch'),
    (16, 7, 3, 'batch'),
]
fc_units = [512]
dropout_p = 0.05406253916379472

class FixedCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        layers = []
        in_ch = 3
        for out_ch, k, p, norm in conv_params:
            layers.append(nn.Conv2d(in_ch, out_ch, kernel_size=k, padding=p))
            if norm == 'batch':
                layers.append(nn.BatchNorm2d(out_ch))
            elif norm == 'layer':
                layers.append(nn.GroupNorm(1, out_ch))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.MaxPool2d(2))
            in_ch = out_ch
        self.feature_extractor = nn.Sequential(*layers)
        with torch.no_grad():
            dummy = torch.zeros(1, 3, 224, 224)
            feat = self.feature_extractor(dummy)
            flat_dim = feat.view(1, -1).size(1)
        fc_layers = []
        in_feat = flat_dim
        for out_feat in fc_units:
            fc_layers.append(nn.Linear(in_feat, out_feat))
            fc_layers.append(nn.ReLU(inplace=True))
            fc_layers.append(nn.Dropout(dropout_p))
            in_feat = out_feat
        fc_layers.append(nn.Linear(in_feat, num_classes))
        self.classifier = nn.Sequential(*fc_layers)
        self.flat_dim = flat_dim

    def forward(self, x, return_features=False):
        x = self.feature_extractor(x)
        x_flat = x.view(x.size(0), -1)
        if return_features:
            return x_flat
        return self.classifier(x_flat)

# Instantiate and load pre-trained CNN
cnn_model = FixedCNN(num_classes).to(device)
optimizer = optim.RMSprop(
    cnn_model.parameters(),
    lr=8.742429747329988e-05,
    weight_decay=0.0008288643868674648
)
criterion = nn.CrossEntropyLoss()

# Train CNN (or assume pre-trained weights are loaded)
epochs = 19
for epoch in range(1, epochs + 1):
    cnn_model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, _, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = cnn_model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    print(f"Epoch {epoch}: Loss = {epoch_loss:.4f}, Acc = {epoch_acc:.2f}%")

# Extract CNN features for training and test sets
cnn_model.eval()
train_cnn_features = []
train_labels = []
test_cnn_features = []
test_labels = []

with torch.no_grad():
    for imgs, _, labels in tqdm(train_loader, desc="Extracting train CNN features"):
        imgs = imgs.to(device)
        features = cnn_model(imgs, return_features=True).cpu().numpy()
        train_cnn_features.append(features)
        train_labels.append(labels.numpy())
    for imgs, _, labels in tqdm(test_loader, desc="Extracting test CNN features"):
        imgs = imgs.to(device)
        features = cnn_model(imgs, return_features=True).cpu().numpy()
        test_cnn_features.append(features)
        test_labels.append(labels.numpy())

train_cnn_features = np.concatenate(train_cnn_features, axis=0)
test_cnn_features = np.concatenate(test_cnn_features, axis=0)
train_labels = np.concatenate(train_labels, axis=0)
test_labels = np.concatenate(test_labels, axis=0)
print("Train CNN features shape:", train_cnn_features.shape)
print("Test CNN features shape:", test_cnn_features.shape)

# Normalize radiomics features for compatibility
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_rad_scaled = scaler.fit_transform(X_train_rad)
X_test_rad_scaled = scaler.transform(X_test_rad)

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Load shared setup (assumed to be executed)
# Variables available: train_cnn_features, test_cnn_features, X_train_rad_scaled, X_test_rad_scaled, train_labels, test_labels

# Concatenate CNN and radiomics features
train_fused_features = np.concatenate([train_cnn_features, X_train_rad_scaled], axis=1)
test_fused_features = np.concatenate([test_cnn_features, X_test_rad_scaled], axis=1)
print("Train fused features shape:", train_fused_features.shape)
print("Test fused features shape:", test_fused_features.shape)

# Train Random Forest on fused features
rf_fusion = RandomForestClassifier(n_estimators=100, random_state=42)
rf_fusion.fit(train_fused_features, train_labels)

# Evaluate on test set
y_pred_fusion = rf_fusion.predict(test_fused_features)
test_acc = accuracy_score(test_labels, y_pred_fusion) * 100
print(f"\nFeature-Level Fusion (Random Forest) Test Accuracy: {test_acc:.2f}%")
print("Classification Report:")
print(classification_report(test_labels, y_pred_fusion))

In [ ]:
import numpy as np
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Load shared setup (assumed to be executed)
# Variables available: cnn_model, train_loader, test_loader, X_train_rad_scaled, X_test_rad_scaled, train_labels, test_labels

# Train Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_rad_scaled, train_labels)

# Get CNN probabilities
cnn_model.eval()
train_cnn_probs = []
test_cnn_probs = []

with torch.no_grad():
    for imgs, _, _ in tqdm(train_loader, desc="Computing train CNN probs"):
        imgs = imgs.to(device)
        outputs = torch.softmax(cnn_model(imgs), dim=1).cpu().numpy()
        train_cnn_probs.append(outputs)
    for imgs, _, _ in tqdm(test_loader, desc="Computing test CNN probs"):
        imgs = imgs.to(device)
        outputs = torch.softmax(cnn_model(imgs), dim=1).cpu().numpy()
        test_cnn_probs.append(outputs)

train_cnn_probs = np.concatenate(train_cnn_probs, axis=0)
test_cnn_probs = np.concatenate(test_cnn_probs, axis=0)

# Get Random Forest probabilities
train_rf_probs = rf_clf.predict_proba(X_train_rad_scaled)
test_rf_probs = rf_clf.predict_proba(X_test_rad_scaled)

# Weighted Averaging (weights based on accuracies: CNN 93%, RF 76%)
w_cnn, w_rf = 0.7, 0.3
train_fused_probs = w_cnn * train_cnn_probs + w_rf * train_rf_probs
test_fused_probs = w_cnn * test_cnn_probs + w_rf * test_rf_probs
y_pred_weighted = np.argmax(test_fused_probs, axis=1)
test_acc_weighted = accuracy_score(test_labels, y_pred_weighted) * 100
print(f"\nDecision-Level Fusion (Weighted Averaging) Test Accuracy: {test_acc_weighted:.2f}%")
print("Classification Report:")
print(classification_report(test_labels, y_pred_weighted))

# Stacking with Logistic Regression
train_stack_features = np.concatenate([train_cnn_probs, train_rf_probs], axis=1)
test_stack_features = np.concatenate([test_cnn_probs, test_rf_probs], axis=1)
meta_classifier = LogisticRegression(multi_class='multinomial', random_state=42)
meta_classifier.fit(train_stack_features, train_labels)
y_pred_stack = meta_classifier.predict(test_stack_features)
test_acc_stack = accuracy_score(test_labels, y_pred_stack) * 100
print(f"\nDecision-Level Fusion (Stacking) Test Accuracy: {test_acc_stack:.2f}%")
print("Classification Report:")
print(classification_report(test_labels, y_pred_stack))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Load shared setup (assumed to be executed)
# Variables available: cnn_model, train_loader, test_loader, num_classes, device, criterion

# Define Hybrid Fusion Model
class HybridFusionModel(nn.Module):
    def __init__(self, cnn_model, radiomics_dim, num_classes):
        super().__init__()
        self.cnn = cnn_model
        for param in self.cnn.parameters():
            param.requires_grad = False  # Freeze CNN weights
        self.cnn_feature_dim = cnn_model.flat_dim
        self.fc_radiomics = nn.Linear(radiomics_dim, 128)
        self.fc_fused = nn.Sequential(
            nn.Linear(self.cnn_feature_dim + 128, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, imgs, rad_features):
        cnn_features = self.cnn(imgs, return_features=True)
        rad_features = torch.relu(self.fc_radiomics(rad_features))
        fused = torch.cat([cnn_features, rad_features], dim=1)
        return self.fc_fused(fused)

# Instantiate model
radiomics_dim = X_train_rad_scaled.shape[1]
hybrid_model = HybridFusionModel(cnn_model, radiomics_dim, num_classes).to(device)
optimizer = optim.Adam(hybrid_model.parameters(), lr=1e-3, weight_decay=1e-4)

# Training loop
epochs = 10
for epoch in range(1, epochs + 1):
    hybrid_model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, rad_features, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}"):
        imgs, rad_features, labels = imgs.to(device), rad_features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = hybrid_model(imgs, rad_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    print(f"Epoch {epoch}: Loss = {epoch_loss:.4f}, Acc = {epoch_acc:.2f}%")

# Evaluate on test set
hybrid_model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for imgs, rad_features, labels in tqdm(test_loader, desc="Testing"):
        imgs, rad_features = imgs.to(device), rad_features.to(device)
        outputs = hybrid_model(imgs, rad_features)
        preds = outputs.argmax(dim=1).cpu().tolist()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().tolist())

test_acc = accuracy_score(all_labels, all_preds) * 100
print(f"\nHybrid Fusion Test Accuracy: {test_acc:.2f}%")
print("Classification Report:")
print(classification_report(all_labels, all_preds))

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Load shared setup (assumed to be executed)
# Variables available: train_cnn_features, test_cnn_features, X_train_rad_scaled, X_test_rad_scaled, train_labels, test_labels

# Concatenate CNN features with radiomics features
train_cascade_features = np.concatenate([train_cnn_features, X_train_rad_scaled], axis=1)
test_cascade_features = np.concatenate([test_cnn_features, X_test_rad_scaled], axis=1)
print("Train cascade features shape:", train_cascade_features.shape)
print("Test cascade features shape:", test_cascade_features.shape)

# Train Random Forest on cascaded features
rf_cascade = RandomForestClassifier(n_estimators=100, random_state=42)
rf_cascade.fit(train_cascade_features, train_labels)

# Evaluate on test set
y_pred_cascade = rf_cascade.predict(test_cascade_features)
test_acc = accuracy_score(test_labels, y_pred_cascade) * 100
print(f"\nCascade Fusion (CNN -> RF) Test Accuracy: {test_acc:.2f}%")
print("Classification Report:")
print(classification_report(test_labels, y_pred_cascade))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Load shared setup (assumed to be executed)
# Variables available: cnn_model, train_loader, test_loader, num_classes, device, criterion

# Define Attention Fusion Model
class AttentionFusionModel(nn.Module):
    def __init__(self, cnn_model, radiomics_dim, num_classes):
        super().__init__()
        self.cnn = cnn_model
        for param in self.cnn.parameters():
            param.requires_grad = False  # Freeze CNN weights
        self.cnn_feature_dim = cnn_model.flat_dim
        self.fc_radiomics = nn.Linear(radiomics_dim, 128)
        self.attention = nn.Sequential(
            nn.Linear(self.cnn_feature_dim + 128, 64),
            nn.Tanh(),
            nn.Linear(64, 1),
            nn.Softmax(dim=1)
        )
        self.fc_fused = nn.Sequential(
            nn.Linear(self.cnn_feature_dim + 128, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, imgs, rad_features):
        cnn_features = self.cnn(imgs, return_features=True)
        rad_features = torch.relu(self.fc_radiomics(rad_features))
        combined = torch.cat([cnn_features, rad_features], dim=1)
        attn_weights = self.attention(combined)
        weighted_features = combined * attn_weights
        return self.fc_fused(weighted_features)

# Instantiate model
radiomics_dim = X_train_rad_scaled.shape[1]
attention_model = AttentionFusionModel(cnn_model, radiomics_dim, num_classes).to(device)
optimizer = optim.Adam(attention_model.parameters(), lr=1e-3, weight_decay=1e-4)

# Training loop
epochs = 10
for epoch in range(1, epochs + 1):
    attention_model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, rad_features, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}"):
        imgs, rad_features, labels = imgs.to(device), rad_features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = attention_model(imgs, rad_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    print(f"Epoch {epoch}: Loss = {epoch_loss:.4f}, Acc = {epoch_acc:.2f}%")

# Evaluate on test set
attention_model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for imgs, rad_features, labels in tqdm(test_loader, desc="Testing"):
        imgs, rad_features = imgs.to(device), rad_features.to(device)
        outputs = attention_model(imgs, rad_features)
        preds = outputs.argmax(dim=1).cpu().tolist()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().tolist())

test_acc = accuracy_score(all_labels, all_preds) * 100
print(f"\nAttention-Based Fusion Test Accuracy: {test_acc:.2f}%")
print("Classification Report:")
print(classification_report(all_labels, all_preds))